In [17]:
from PyPDF2 import PdfReader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter # to split the lines
from langchain_community.vectorstores import FAISS

In [18]:
OPENAI_API_KEY = "Give your custom OpenAI key"

In [19]:
pdfreader = PdfReader('elections.pdf')

In [20]:
from typing_extensions import Concatenate

raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [21]:
len(raw_text)

5690

In [22]:

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 200,
    chunk_overlap  = 50,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [23]:
len(texts)

37

In [24]:

embeddings = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY)

In [25]:
document_search = FAISS.from_texts(texts, embedding=embeddings)

In [26]:
document_search

In [27]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [29]:
chain = load_qa_chain(OpenAI(openai_api_key = OPENAI_API_KEY), chain_type="stuff")

In [30]:
query = "where is election commision of India is located?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

C:\Users\user\anaconda3\envs\lang_env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


' Nirvachan Sadan, Ashoka Road, New Delhi-110001'

In [39]:
query = "Howmany observers are present?"
docs = document_search.similarity_search(query)

results = chain.invoke({
    'question': "Howmany observers are present?",
    'input_documents': docs
})

In [41]:
results['output_text']

' 172 Observers are present.'